<center>
<h4>Universidad Nacional de Córdoba - Facultad de Matemática, Astronomía, Física y Computación</h4>
<h3>Diplomatura en Ciencia de Datos, Aprendizaje Automático y sus Aplicaciones</h3>
</center>

<h1> Práctico </h1>
<h3> Análisis y Visualización de Datos - 2020 </h3>

Durante este práctico vamos a trabajar sobre el dataset [Human Freedom Index 2018](https://www.cato.org/human-freedom-index-new) de el instituto Cato. Este índice mide en detalle lo que entendemos como libertad, utilizando 79 indicadores de libertad personal y económica en distintos aspectos, hasta obtener un hermoso numerito del 1 al 10.

Las variables más importantes sobre las que trabaja el dataset son:

* Rule of Law
* Security and Safety
* Movement
* Religion
* Association, Assembly, and Civil Society
* Expression and Information
* Identity and Relationships
* Size of Government
* Legal System and Property Rights
* Access to Sound Money
* Freedom to Trade Internationally
* Regulation of Credit, Labor, and Business

Nosotros centrarermos nuestro análisis en variables relacionadas a *Identity and Relationships* en paises de Latinoamérica, y los compararemos con las estadísticas globales. La pregunta a responder es simple: **¿Qué niveles de libertad se viven en Latinoamérica, especificamente en cuanto libertades de indentidad?**. Sin embargo, para hacer un análisis de los datos tenemos que platear también estas sub preguntas:

1. ¿Qué significa tener un puntaje de 4.5? Hay que poner los puntajes de la región en contexto con los datos del resto del mundo.
2. ¿Cuál es la tendencia a lo largo de los años? ¿Estamos mejorando, empeorando?
3. En este estudio, la libertad se mide con un estimadores principal: *hf_score* que hace referencia a Human Freedom, que a su vez está calculado en base a dos otros estimadores *ef_score*, para Economic Freedom y *pf_score*, para Personal Freedom. Estos tres estimadores, ¿se relacionan de la misma manera con la libertad de identidad?

Inicialmente, en toda exploración de datos tenemos muy poca información a priori sobre el significado de los datos y tenemos que empezar por comprenderlos. Les proponemos los siguientes ejercicios como guía para comenzar esta exploración.

In [ ]:
import matplotlib.pyplot as plt
import numpy
import pandas
import seaborn as sns

In [ ]:
sns.__version__

Si un archivo está disponible en la web, podemos leerlo con pandas utilizando su URL sin necesidad de descargarlo.

In [ ]:
dataset = pandas.read_csv('https://object.cato.org/sites/cato.org/files/human-freedom-index-files/human-freedom-index-2019.csv')
dataset.shape

In [ ]:
dataset.columns  # Way too many columns!

Por suerte las columnas tienen un prefijo que nos ayuda a identificar a qué sección pertenecen. Nos quedamos sólo con las que comienzan con *pf_indentity*, junto con otras columnas más generales

In [ ]:
score_cols = [col for col in dataset.columns if 'pf_identity' in col] + [
    'pf_score', # Personal Freedom (score)
    'pf_rank', # Personal Freedom (rank)
    'ef_score', # Economic Freedom (score)
    'ef_rank', # Economic Freedom (rank)
    'hf_score', # Human Freedom (score)
    'hf_rank', # Human Freedom (rank)
]

important_cols = ['year', 'ISO_code', 'countries', 'region'] + score_cols

In [ ]:
dataset[important_cols]

Si analizamos los tipos, vemos que casi todas las columnas se leen como objetos debido a que los valores nulos están marcados con `-`. Los vamos a eliminar y volveremos a convertir las columnas con puntajes a tipo numérico.

In [ ]:
dataset.dtypes

In [ ]:
dataset = dataset[important_cols].replace('-', numpy.nan)
for score_col in score_cols: 
    dataset[score_col] = pandas.to_numeric(dataset[score_col])
dataset

# Parte 1

Luego de las clases del primer fin de semana, ya tenemos las herramientas para responder las siguientes preguntas:

# 1. Estadísticos descriptivos

1. Para comenzar con un pantallazo de los datos, calcular el rango de las variables.

In [ ]:
#cantidad de paises 162.. creo que existen mas de 194 por lo que en el análisis hay muchos países que quedaron afuera
print('Cantidad de países',len(dataset['countries'].unique()))

In [ ]:
#pantallazo del rango de las variables
dataset.describe()

Podemos decir rapidamente que:
- la puntuación de la libertad va entre 0 y 10
- los años van entre 2008 a 2017
- que se analizaron los 162 países en los 10 años
- hay 14 puntajes, 
- hay 2 puntajes mas importantes que son libertad económica y personal
- hay 1 puntaje con la libertad humana que es promedio de estás 2 últimas
- 3 columnas con el ranking de los 3 puntajes mas relevantes

In [ ]:
#se verifica que la libertad humana es el promedio entre la libertad personal y la económica
# hf_score = (pf_score + ef_score)/2
temp_dataset = dataset[ (((dataset['pf_score']+dataset['ef_score'])/2)-dataset['hf_score']).abs()>0.01 ]
if temp_dataset.empty:
    print("se verifica que hf_score = (pf_score + ef_score)/2")
else:
    print("hay {} que no vefica que hf_score = (pf_score + ef_score)/2".format(len(temp_dataset)))
    

2. Para cada país, tenemos varias realizaciones para cada variable *pf_identity* y *hf_score*. Si queremos comparar un país con otro, ¿cuál es la manera adecuada de hacerlo? Por ejemplo, ¿nos quedamos con un único valor? ¿o comparamos todos los valores? ¿usamos el promedio? ¿usamos la mediana? 

In [ ]:
#analizo la libertad mundial, media, mediana, dev std, máxima y minima
# también agrego a Argentina
pandas.options.display.float_format = '{:,.2f}'.format
i_max=dataset.groupby('year')['hf_score'].idxmax()
i_min=dataset.groupby('year')['hf_score'].idxmin()
dataset_byyear = pandas.DataFrame()

# agrego media
dataset_byyear['hf_score_mean'] = dataset.groupby('year').mean()['hf_score']
dataset_byyear['pf_score_mean'] = dataset.groupby('year').mean()['pf_score']

#agrego mediana
dataset_byyear['hf_score_median'] = dataset.groupby('year').median()['hf_score'] #por defecto saltea NaN values
dataset_byyear['pf_score_median'] = dataset.groupby('year').median()['pf_score'] #por defecto saltea NaN values

#agrego desviación estandar
dataset_byyear['hf_score_std'] = dataset.groupby('year').std()['hf_score']
dataset_byyear['pf_score_std'] = dataset.groupby('year').std()['pf_score']


#agrego valores máximos y mínimos, incluye el país en esa posición
dataset_byyear[['country_max','hf_score_max']] = dataset[['year','countries','hf_score']].loc[i_max].set_index('year')
dataset_byyear[['country_min','hf_score_min']] = dataset[['year','countries','hf_score']].loc[i_min].set_index('year')

#datos de argentina
dataset_byyear[['hf_score_argentina']] = dataset[dataset['countries']=='Argentina'][['year','hf_score']].set_index('year')
dataset_byyear[['pf_score_argentina']] = dataset[dataset['countries']=='Argentina'][['year','pf_score']].set_index('year')


dataset_byyear

Considero que los datos deben analizarse en el tiempo, agrupar valores sin considerar la fecha no tiene ningún sentido para tratar de identificar si un país está mejor que otro.
Para esto podríamos analizar su valor medio actual y su tendencia en los últimos años.
Mas adelante analizaré si hay correlación entre distintas libertades para distintos países.  
En la tabla de arriba podemos decir que Nueva Zelanda es uno de los mejores países y Siria uno de los peores con bastante segurdidad.  
También que Argentina está cerca de la media mundial. Pero considero que con valores hasta el 2017 hoy no podemos afirmar eso.

3. Obtener media, mediana y desviación estándar de las variables *pf_identity* y *hf_score* en el mundo y compararla con la de Latinoamérica y el caribe. Usar la respuesta del punto anterior para justificar si la comparación es válida o no.

In [ ]:
pandas.__version__

In [ ]:
#datos por region, se usa pivot

#calculo mas métricas regionales
regions_mean = dataset.groupby(['year','region']).mean()
regions_median = dataset.groupby(['year','region']).median()
regions_std = dataset.dropna().groupby(['year','region']).std()

#armo dataframe con puntajes por region
regions = pandas.DataFrame()
regions[['hf_score_mean','pf_score_mean']] = regions_mean[['hf_score','pf_score']]
regions[['hf_score_median','pf_score_median']] = regions_median[['hf_score','pf_score']]
regions[['hf_score_std','pf_score_std']] = regions_std[['hf_score','pf_score']]
# regions.columns = pandas.MultiIndex.from_product([['mean','median','std'],['hf_score','pf_score']])
regions.reset_index(inplace=True)

# Se selecciona la region Latino America y Caribe
df_LAC = regions.pivot(index='year',columns='region',values=regions.columns.values[2:])\
.xs('Latin America & the Caribbean', level='region', axis=1)

# Se une con los datos globales
df_LAC=df_LAC.join(dataset_byyear[dataset_byyear.columns[:6]],lsuffix='_LA&C', rsuffix='_G', how='inner', sort=True)
df_LAC.sort_index(axis=1,inplace=True)
df_LAC.columns = pandas.MultiIndex.from_product([['mean','median','std'],['hf_score','pf_score'],['World','LA&C']])
df_LAC


In [ ]:
# latin_id = 'Latin America & the Caribbean'
# dataset_byyear[latin_id]=regions_ef_pf[['hf_score',latin_id]]

In [ ]:
#copio latino america al dataframe
latin_id = regions.columns[3]
dataset_byyear[latin_id]=regions[latin_id]


In [ ]:
# para comprobar que hace pandas cuando calcula la mediana
# temp=dataset[ (dataset['year']==2008) & (dataset['hf_score'].notna()) ].sort_values('hf_score')
# temp['hf_score'].iloc[int((len(temp)/2))]

In [ ]:
import plotly.graph_objects as go
import plotly

In [ ]:
plotly.__version__

In [ ]:
x = dataset_byyear.index
y=  dataset_byyear['hf_score_mean']
ymax = dataset_byyear['hf_score_max']
ymin = dataset_byyear['hf_score_min']
ystd_up=  y + dataset.groupby('year').std()['hf_score']
ystd_down=  y - dataset.groupby('year').std()['hf_score']

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y, 
                         name='Media mundial', mode='lines', line=dict(color='royalblue', width=4))
             )

fig.add_trace(go.Scatter(x=x, y=dataset_byyear['hf_score_argentina'], 
                         name='Libertad en Argentina', mode='lines', line=dict(color='orange', width=4))
             )

fig.add_trace(go.Scatter(x=x, y=dataset_byyear[latin_id], 
                         name=latin_id, mode='lines', line=dict(color='brown', width=4))
             )

fig.add_trace(go.Scatter(x=x, y=ymax, text=dataset_byyear['country_max'], textposition= 'top center',
                         name='Máxima Libertad', mode='markers', line=dict(color='green', width=2)
                        #,visible='legendonly'
                        )
             )

fig.add_trace(go.Scatter(x=x, y=ymin, text=dataset_byyear['country_min'], textposition= 'top center',
                         name='Mínima Libertad', mode='markers', line=dict(color='red', width=2)
                        #,visible='legendonly'
                        )
             )

fig.add_trace(go.Scatter(x=x, y=ystd_down, 
                         name='Std down', mode='lines', line=dict(width=2)
                        ,visible='legendonly')
             )

fig.add_trace(go.Scatter(x=x, y=ystd_up, 
                         name='Std up', mode='lines', line=dict(width=2)
                        ,visible='legendonly')
             )

# fig.add_trace(go.Scatter(x=dataset['year'], y=dataset['hf_score'], text=dataset['countries'], 
#                          name='Todos los países', mode='markers', marker_size=4
#                         ,visible='legendonly')
#              )

fig.add_trace(go.Box(
                    x=dataset['year'], y=dataset['hf_score'], text=dataset['countries'], 
                    name="Todos los países",
#                     jitter=0.3,
#                     pointpos=-1.8,
                    boxpoints='all', # represent all points
                    marker_color='rgb(17,80,89)',
                    line_color='rgb(17,80,89)'
                    ,boxmean=True
                    ,visible='legendonly'
                    )
             )


# fig.update_traces(textposition='top center')

fig.update_layout(
    title=go.layout.Title(
        text="Libertad Mundial / Argentina / Latino América",
        xref="paper",
        x=0.1,
        font_size=18,
    ),
    xaxis=go.layout.XAxis(
        title=go.layout.xaxis.Title(
            text="Año",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    ),
    yaxis=go.layout.YAxis(
        title=go.layout.yaxis.Title(
            text="Puntaje",
            font=dict(
                family="Courier New, monospace",
                size=18,
                color="#7f7f7f"
            )
        )
    )
)

fig.show()

- Se puede observar que la libertad mundial y de latino América son similares.
- La libertad en Argentina tiene un quiebre en la tendencia a partir de 2014 donde en el 2018 se recuperó hasta la media mundial
- El gráfico permite ocultar y visualizar distintas curvas haciendo click en la leyenda de la derecha
- Se puede activar el diagrama de caja de todos los países, se puede hacer zoom en el año y extremo deseado si nos paramos con el cursor del mouse se pueden observar valores exactos de cada medida  para una visualización panorámica considero que seaborn es mucho mas sencillo de usar y cumple perfectamente las espectativas
- Para gráficos donde se quiere ver el detalle, plotly se adapta mejor 
- Si se activa "Todos los países" se puede ver a Siria como Outlier, de ninguna manera sacaría a Siria cuando se está hablando de libertad humana, hasta es preocupante!
- Otra observación es que la tendencia tanto en latino América y mundial está en descenso.. se podría calcular la pendiente,  pero no creo que aporte demasiado, ya que decir que en x años no vamos a tener mas libertad sería absurdo
- Se podría estimar cuando podría haber una Revolución pero habría que entender a que valor de hf_score podría ocurrir.

4. ¿Tiene sentido calcular la moda?
No tiene sentido analizar la moda, ya que es para valores categóricos. 

5. Cómo pueden sanearse los valores faltantes?
No lo haría por tratarse de valores de libertad, pero se podría interpolar o extrapolar con puntos de otro año si es que los tuviese.

  6. ¿Encuentra outliers en estas dos variables? ¿Qué método utiliza para detectarlos? Los outliers, ¿son globales o por grupo? ¿Los eliminaría del conjunto de datos?
  

Usando el box plot encuentro a Siria como outlier, plotly usa los cuartiles y la mediana: valores inferiores a Q1–1.5·RIC o superiores a Q3+1.5·RIC.
Habría que leer un poco sobre este país y ver si tienen realmente un problema de libertad, pero nunca quitaría este valor

In [ ]:
# un ejemplo para encontrar outliers en el año 2017, usando la media y 2.5 x desviación estandar
dataset2017 = dataset[dataset['year']==2017] #selecciono el año
mean = dataset2017['hf_score'].mean() # media
std = dataset2017['hf_score'].std() # desviación estandar
filtro = (dataset2017['hf_score']-mean).abs()>=(2.5*std) # filtro valores con distancia mayor a 2.5*std desde la media
filtro_nan = dataset2017['hf_score'].notna() # filtro NaN
dataset2017[(filtro)&(filtro_nan)]


En la tabla de arriba se muestran Outliers en el año 2017, usando la media y 2.5 x desviación estandar

## 2. Agregación de datos

1. Grafiquen la media de la variable *pf_identity* y *hf_score* a través de los años.

Pista: hay gráficos de seaborn que permiten generar visualizaciones para cada valor de una variable categórica, en este caso, las distintas regiones.

Sólo por curiosidad, graficar la tendencia de *hf_score* y *ef_score* a través de los años. ¿Tienen alguna hipótesis para este comportamiento?

In [ ]:
# se grafica la libertad económica y personal mundial
sns.set_context(context='talk', font_scale=0.8, rc={"lines.linewidth": 3})
plt.figure(figsize=(10,6))
sns.lineplot(data=dataset,
                 x='year', y='ef_score', label='Libertad económica', color='Orange',
                 estimator = 'mean',
                 ci = 'sd',
                 n_boot = 1000 #solo se usa para ci 0 a 100%
                )

sns.lineplot(data=dataset,
                 x='year', y='pf_score', label='Libertad personal', color='Green',
                 estimator = 'mean',
                 ci = 'sd',
                 n_boot = 1000 #solo se usa para ci 0 a 100%
                )



plt.legend()
plt.legend(bbox_to_anchor=(1.4, 0.9), loc=0, borderaxespad=0, shadow=True)
plt.title("Variación de las Libertades en el mundo")
#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
plt.ylabel('Puntaje')
plt.xlabel('Año')
sns.despine()

2.2. Realicen los mismos gráficos, pero separando por regiones (Cada variable en un gráfico distinto, sino no se ve nada). ¿La tendencia observada, es la misma que si no dividimos por regiones?

In [ ]:
import plotly.express as px

df = dataset.groupby(['year','region'], as_index=False).mean()
fig = px.line(df, x='year', y='ef_score', color="region", line_group="region", hover_name="region", line_shape='spline')

fig.update_layout(
    title="Libertad económica por Regiones",
    xaxis_title="Año",
    yaxis_title="Libertad económica"
                )
fig.show()

In [ ]:
df = dataset.groupby(['year','region'], as_index=False).mean()
fig = px.line(df, x='year', y='pf_score', color="region", line_group="region", hover_name="region", line_shape='spline')

fig.update_layout(
    title="Libertad personal por Regiones",
    xaxis_title="Año",
    yaxis_title="Libertad personal"
                )
fig.show()

Al igual que en el gráfico de la libertad mundial, por región vemos que en general la libertad personal tiende a disminuir en todas la regiones, en lo económico tiene un leve alza.  
Nota, Ploty express permite hacer algunos gráficos bastantes rápidos.

2.3. Si lo consideran necesario, grafiquen algunos países de Latinoamerica para tratar de explicar la tendencia de la variable *pf_identity* en la región. ¿Cómo seleccionarion los países relevantes a esa tendencia?

In [ ]:
dataset_world = dataset.groupby('year').mean()[['ef_score','pf_score']]
dataset_world

In [ ]:
countries = ['Argentina', 'Brazil', 'Mexico', 'Bolivia',
             'Uruguay', 'Peru', 'Colombia', 'Venezuela']
df=dataset[dataset['countries'].isin(countries)].copy()
fig_pf = px.line(df, x='year', y='pf_score', color="countries",line_group="countries", hover_name="countries"
              ,line_shape='spline'
             )

fig_pf.update_traces(visible='legendonly')

fig_pf.data[0].visible=True
fig_pf.data[-2].visible=True
fig_pf.data[-1].visible=True

fig_pf.add_trace(go.Scatter(x=dataset_world.index, y=dataset_world['pf_score'], 
                         name='Libertad Personal mundial', mode='lines', line=dict(color='gray', width=5))
             )

fig_pf.update_layout(
    title=go.layout.Title(
        text="Libertad personal en algunos paises de Latinoamérica",
        xref="paper",
        x=0.1,
        font_size=18,
    ),
    xaxis_title="Año",
    yaxis_title="Puntaje",
    legend_title_text='Países'
)


fig_pf.show()



In [ ]:
countries = ['Argentina', 'Brazil', 'Mexico', 'Bolivia',
             'Uruguay', 'Peru', 'Colombia', 'Venezuela']
df=dataset[dataset['countries'].isin(countries)].copy()

fig_ef = px.line(df, x='year', y='ef_score', color="countries",line_group="countries", hover_name="countries"
              ,line_shape='spline'
             )

fig_ef.update_traces(visible='legendonly')

fig_ef.data[0].visible=True
fig_ef.data[-2].visible=True
fig_ef.data[-1].visible=True

fig_ef.add_trace(go.Scatter(x=dataset_world.index, y=dataset_world['ef_score'], 
                         name='Libertad Económica mundial', mode='lines', line=dict(color='gray', width=5)
                        ) 
             )


fig_ef.update_layout(
    title=go.layout.Title(
        text="Libertad Económica en algunos paises de Latinoamérica",
        xref="paper",
        x=0.1,
        font_size=18,
    ),
    xaxis_title="Año",
    yaxis_title="Puntaje",
    legend_title_text='Países'
)


fig_ef.show()


Los gráficos de arriba se puede comprar algunos países de latino América, con el gráfico se puede interactuar habilitando los países que se desean observar y ocultando lo que no se desean ver.  
Hasta el 2017 Argentina no se parece a Venezuela.  
Uruguay parece el mejor país que nos rodea, tanto en lo económico como en lo personal.

In [ ]:
# import plotly.offline as pltly
# pltly.plot(fig_pf, filename='pf_score.html',auto_open=False)
# pltly.plot(fig_ef, filename='ef_score.html',auto_open=False)

## 3. Distribuciones
  1. Graficar en un mismo histograma la distribución de la variable *pf_identity* en global, y en Latinoamérica y el caribe. Repetir para la variable *hf_score*. ¿Visualmente, a qué tipo de distribución corresponde cada variable? ¿Es correcto utilizar todos el conjunto de valores disponibles para esa region en estos gráficos?


In [ ]:
plt.figure(figsize=(15,14))
plt.suptitle('Histogramas distribución de la Libertad Personal', y=1.005)

f = dataset['region']=='Latin America & the Caribbean'
with sns.axes_style("white"):
    
    sp1=plt.subplot(2,1,1)
    sns.distplot(dataset['hf_score'],kde=True, norm_hist=True, bins=10, color='green'
                ,label = 'Mundial'
                , ax=sp1
                )
    sns.distplot(dataset[f]['hf_score'], kde=True, norm_hist=True, bins=10, color='orange'
                ,label='América Latina y Caribe'
                ,ax=sp1
                )

    sp1.set_xlabel('Libertad personal', weight='bold')
    sp1.set_ylabel('Frecuencia normalizada')
    sp1.set_xlim(2,10)
    sp1.legend(shadow=True)
    sns.despine()

    sp2 = plt.subplot(2,1,2)
    sns.distplot(dataset['ef_score'],kde=True, norm_hist=True, bins=10, color='green'
                ,label = 'Mundial'
                , ax=sp2
                )
    sns.distplot(dataset[f]['ef_score'], kde=True, norm_hist=True, bins=10, color='orange'
                ,label='América Latina y Caribe'
                ,ax=sp2
                )

    sp2.set_xlabel('Libertad económica', weight='bold')
    sp2.set_ylabel('Frecuencia normalizada')
    sp2.set_xlim(2,10)
    sp2.legend(shadow=True)
    sns.despine()    
    

    #sp2.legend(bbox_to_anchor=(1.5, 0.9), loc=0, borderaxespad=0, shadow=True)

plt.show()

La libertad económica pareciera tener distribución normal. Aunque hay países como Venezuela fuera de los normal.   
Es correcto tomar todos los valores si queremos ver los 10 años en conjunto.

## 4. Correlaciones y Relaciones

En este ejercicio queremos responder a las preguntas

* Las libertades personales y económicas, ¿van siempre de la mano?
* ¿Cómo se relacionan ambas con las libertades respectivas a las relaciones personales?

Para ello, analizaremos las correlaciones entre las variables pf_identity, pf_score y ef_score. 

Como pf_indentity contribuye al cálculo de pf_score esperamos hallar algún grado de correlación. Lo contrario podría ocurrir con ef_score. 

1. ¿Qué conclusiones puede sacar de un gráfico pairplot de estas tres variables? ¿Es adecuado para los valores de pf_identity? ¿Por qué?
2. Graficar la correlación (visual) entre pf_identity y pf_score; y entre pf_identity y ef_score. Analizar el resultado, ¿se pueden sacar conclusiones? Tengan en cuenta que como pf_identity es el resultado de un promedio, sólo toma algunos valores. Es, en la práctica, discreta, y eso afecta al tipo de gráfico que podemos usar.
3. Convertir estas variables en categóricas, es decir, a partir de pf_indentity generar otra variable pf_identity_segment que tome los valores `high`, `medium` y `low`. Pueden hacerlo con una función escrita por ustedes, o usando alguna función de pandas como `pandas.cut` o `pandas.dcut`. Repetir para ef_score y pf_score. El criterio para decidir qué intervalos de valores corresponden a cada categoría tienen que decidirlo ustedes, pueden usar los estadísticos mediana y los cuartiles.
4. Graficar la correlación (visual) entre estas tres variables categoricas usando gráficos de calor (heatmaps). Note: van a necesitar 3 gráficos distintos, porque en cada uno podemos incluir sólo 2 variables.



In [ ]:
with sns.axes_style("white"):
    g = sns.PairGrid(data=dataset, vars=['pf_identity', 'pf_score', 'ef_score'], height=4, aspect=1)
    g.map_lower(sns.regplot, ci=95, scatter_kws={"s": 2}, line_kws={'color':'g'}) #Intervalo de Confianza = 95%
    g.map_diag (sns.distplot, kde=False, norm_hist=True)
    g.map_upper(sns.kdeplot, n_levels=10, shade=True, shade_lowest=False)
    
plt.show()

# Parte 2

Luego del segundo fin de semana de clase, podemos revisitar nuestro trabajo anterior y completarlo respondiendo a las siguientes preguntas:

## 3. Distribuciones

2. Realizar una prueba de Kolmogorov-Smirnof para comprobar analíticamente si estas variables responden la distribución propuesta en el ejercicio anterior. Hint: podés usar https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.kstest.html, pero hay que tener en cuenta que si la distribución es "norm", entonces va a comparar los datos con una distribución normal con media 0 y desviación estándar 1. Se puede utilizar la distribución sobre todos los datos o sólo sobre Latinoamérica.

## 4. Correlaciones

3. Calcular algún coeficiente de correlación adecuado entre los dos pares de variables, dependiendo de la cantidad de datos, el tipo de datos y la distribución de los mismo. Algunas opciones son: coeficiente de pearson, coeficiente de spearman, coeficientes de tau y de kendall. Interpretar los resultados y justificar si las variables están correlacionadas o no. 
4. [Opcional] Analizar la correlación entre la region y el pf_score (y/o el ef_score); y entre la region y el pf_identity. Considerar que como la variable *region* es ordinal, debe utilizarse algún tipo de test. Explicar cuáles son los requisitos necesarios para la aplicación de ese test. (Si no se cumplieran, se pueden agregar algunos datos para generar más registros). Genere nuevas variables categóricas ordinales para calcular la correlación Tau de Kendal y genere una tabla de contingencia con esas nuevas variables.

Además de completar estos puntos faltantes, luego de haber visitado los conceptos de percepción visual y comunicación efectiva, están en condiciones de reveer los gráficos realizados y evaluar si pueden ser mejorados. Para ello, puede hacerse las siguientes preguntas:

* ¿Están utilizando el tipo de gráfico adecuado para cada tipo de variable?
* Los gráficos, ¿son legibles?
* Los gráficos generados, ¿responden a las preguntas mostrando un patrón claro? En caso de que no, ¿podemos filtrar los datos para que el patrón sea más evidente? ¿o agruparlos de manera distinta? ¿o cambiar el tipo de gráfico?

In [ ]:
dataset.head(2)